In [ ]:
# df.repartition(2).select(F.spark_partition_id().alias("pid")).collect()

# Preparation

In [ ]:
import sys

from collections import namedtuple
from typing import NamedTuple, Callable, TypeVar
from enum import StrEnum, verify, UNIQUE
from dataclasses import dataclass
from random import randrange

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame, Column

## Spark Configuration

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("graphs")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "/tmp/warehouse")
        .getOrCreate()
)
sc = spark.sparkContext
sc.setCheckpointDir("/tmp/plan/checkpoint")

## Miscellaneous Graph Functions and Classes

In [ ]:
class Graph(NamedTuple):
    """The `Graph` class is the central class to express graphs"""
    edges: DataFrame
    vertices: DataFrame

In [ ]:
@dataclass
class Scope:
    """The `Scope` class abstarcts away inward, outward and all neighbours of a vertex"""
    @verify(UNIQUE)
    class Direction(StrEnum):
        IN = "IN"
        OUT = "OUT"
        INOUT = "INOUT"

    inward: DataFrame
    outward: DataFrame
    all_nbr: DataFrame

    def __init__(self, edges: DataFrame):
        def __wrapper(id: str, nbr: str, dir: 'Scope.Direction'):
            return (
                edges_df
                    .withColumnRenamed(id, "id")
                    .withColumnRenamed(nbr, "nbr")
                    .withColumn("dir", F.lit(dir))
                    .select("id", "nbr", "weight", "dir")
            )
        self.inward = __wrapper("dst", "src", Scope.Direction.IN)
        self.outward = __wrapper("src", "dst", Scope.Direction.OUT)
        self.all_nbr = self.outward.union(self.inward)

    def __call__(self, dir: 'Scope.Direction') -> DataFrame:
        match dir:
            case Scope.Direction.IN:
                return self.inward
            case Scope.Direction.OUT:
                return self.outward
            case Scope.Direction.INOUT:
                return self.all_nbr
            case _:
                raise ValueError(f"Unknown {dir}")

    def __iter__(self):
        yield self.inward
        yield self.outward
        yield self.all_nbr
        

In [ ]:
def graph(disjoint_sets_number: int = 1000, disjoint_set_vertices: int = 500, disjoint_set_edges: int = 1000) -> Graph:
    """Generates a graph
        Parameters:
            disjoint_sets_number (int) : total number of disjoint sets in the graph
            disjoint_set_elements (int) : total number of vertices in a disjoint set in the graph
            disjoint_set_edges (int) : total number of edges in a disjoint set in the graph

        Returns:
            Graph (Graph): a graph according to the specifications
    """
    start = 0
    end = disjoint_set_vertices
    
    disjoint_set_step = max(disjoint_set_vertices, 1000)
    edges = []
    for x in range(disjoint_sets_number):
        edges += [ (x, x + 1, randrange(10)) for x in range(start, end) ] + [ (randrange(start, end), randrange(start, end), randrange(10)) for _ in range(disjoint_set_edges) ]
        start += disjoint_set_step
        end += disjoint_set_step

    edges_df = (
        spark
            .createDataFrame(edges)
            .toDF("src", "dst", "weight")
    )

    vertices = set()

    for x, y, _ in edges:
        vertices.add(x)
        vertices.add(y)

    vertices_df = (
        spark
            .createDataFrame([(x,) for x in vertices])
            .toDF("id")
    )
    return Graph(edges_df, vertices_df)

In [ ]:
def small_graph() -> Graph:
    """Generates a small graph for tests"""
    verties = [ (x,) for x in range(1, 7) ]
    verties_df = (
        spark
            .createDataFrame(verties)
            .toDF("id")
    )
    edges = [ (1, 2, 1), (1, 3, 5), (2, 3, 1), (3, 4, 1), (4, 5, 1), (6, 6, 1) ]
    edges_df = (
        spark
            .createDataFrame(edges)
            .toDF("src", "dst", "weight")
    )
    return Graph(edges_df, verties_df)

In [ ]:
def get_min_dist_init_df(values: DataFrame, start: int = 1) -> DataFrame:
    return values.withColumn("dist",
        F.when(
            col("id") == F.lit(start),
            F.lit(0)
        )
        .otherwise(F.lit(sys.maxsize // 2))
    )

In [ ]:
from datetime import datetime

V = TypeVar("V")
def timer(handler: Callable[[], V]):
    """Timer"""
    start = datetime.now()
    result = handler()
    print(datetime.now() - start)
    return result

# Connected Components

## Pregel

### Implementation

In [ ]:
def pregel_cc(edges: DataFrame, values: DataFrame, steps: int):
    result_df = values
    for i in range(steps):
        result_df = (
            edges.join(result_df, col("src") == col("id"))
                .select(col("dst").alias("id"), col("value").alias("message"))
                .groupby(col("id")).agg(F.min("message").alias("message"))
                .join(result_df, "id", "right")
                .select("id", F.least("message", "value").alias("value"))
        )
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
edges_df.printSchema()

In [ ]:
vertices_df.printSchema()

In [ ]:
pregel_cc_df = vertices_df.withColumn("value", col("id"))
pregel_cc_df = pregel_cc(edges_df, pregel_cc_df, 5)

In [ ]:
pregel_cc_df.show()

#### Large Graph

In [ ]:
edges_df, vertices_df = graph()

In [ ]:
edges_df.printSchema()

In [ ]:
vertices_df.printSchema()

In [ ]:
pregel_cc_df = vertices_df.withColumn("value", col("id"))

In [ ]:
pregel_cc_df = timer(lambda: pregel_cc(edges_df, pregel_cc_df, 15))

The `cc_df` DataFrame is large, so only the total number of connected components is displayed

In [ ]:
pregel_cc_df.groupBy("value").count().count()

## GraphLab

### Implementation

In [ ]:
def graphlab_cc(scope: DataFrame, values: DataFrame, steps: int):
    result_df = values
    for i in range(steps):
        result_nbr_df = (
            scope.alias("s")
                .join(result_df.alias("r"), col("s.nbr") == col("r.id"))
                .groupBy(col("s.id"))
                .agg(F.min("cc").alias("acc_cc"))
        )

        result_df = (
            result_nbr_df.alias("a")
                .join(result_df.alias("r"), "id", "right")
                .select("id", F.least(col("r.cc"), col("a.acc_cc")).alias("cc"))
        )

        if i % 5 == 0:
            result_df = result_df.checkpoint()
    return result_df

### Testing

#### Small Graph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope_df = Scope(edges_df)

In [ ]:
gl_cc_result_df = vertices_df.withColumn("cc", col("id"))

In [ ]:
gl_cc_result_df = graphlab_cc(scope_df.all_nbr, gl_cc_result_df, 5)

In [ ]:
gl_cc_result_df.show()

In [ ]:
gl_cc_result_df.groupBy("cc").count().count()

#### Large Graph

In [ ]:
edges_df, vertices_df = graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope_df = Scope(edges_df)

In [ ]:
gl_cc_result_df = vertices_df.withColumn("cc", col("id"))

In [ ]:
gl_cc_result_df = timer(lambda: graphlab_cc(scope_df.all_nbr, gl_cc_result_df, 8))

In [ ]:
gl_cc_result_df.groupBy("cc").count().count()

## PowerGraph

### Implementation

In [ ]:
def powergraph_cc(scope: DataFrame, values: DataFrame, steps: int):
    result_df = values

    for i in range(steps):
        gather_df = (
            scope.where("id != nbr").alias("s")
                .join(result_df.alias("r"), col("r.id") == col("s.nbr"))
                .select(col("s.id"), col("cc").alias("nbr_cc"))
        )
        sum_df = (
            gather_df
                .groupBy("id")
                .agg(F.min("nbr_cc").alias("acc_cc"))
        )
        apply_df = (
            sum_df
                .join(result_df, "id", "right")
                .select(col("id"), F.least("acc_cc", "cc").alias("cc"))
        )
        scatter_df = apply_df
        result_df = scatter_df
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
pg_cc_result_df = vertices_df.select(col("id"), col("id").alias("cc"))
pg_cc_result_df = powergraph_cc(scope.all_nbr, pg_cc_result_df, 5)

In [ ]:
pg_cc_result_df.show()

#### Large Graph

In [ ]:
edges_df, vertices_df = graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
pg_cc_result_df = vertices_df.select(col("id"), col("id").alias("cc"))
pg_cc_result_df = timer(lambda: powergraph_cc(scope.all_nbr, pg_cc_result_df, 8))

In [ ]:
pg_cc_result_df.groupBy("cc").count().count()

# Single Source Shortest Path

## Pregel

### Implementation

In [ ]:
def pregel_min_dist(edges: DataFrame, values: DataFrame, steps: int):
    result_df = values
    for i in range(steps):
        result_df = (
            edges.join(result_df, col("src") == col("id"))
                .select(col("dst").alias("id"), F.expr("dist + weight").alias("message"))
                .groupby(col("id")).agg(F.min("message").alias("message"))
                .join(result_df, "id", "right")
                .select("id", F.least("message", "dist").alias("dist"))
        )
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
min_dist_df = get_min_dist_init_df(vertices_df, start=1)
min_dist_df = pregel_min_dist(edges_df, min_dist_df, 5)

In [ ]:
min_dist_df.show()

#### Large Graph

In [ ]:
disjoint_set_vertices=500
edges_df, vertices_df = graph(disjoint_set_vertices)

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
min_dist_df = get_min_dist_init_df(vertices_df, start=1)
min_dist_df = timer(lambda: pregel_min_dist(edges_df, min_dist_df, 12))

The node `1` has paths to only the nodes from the same disjoint set, the other disjoint sets are not accessible, hence the min distance to all of them is `INF (sys.maxsize)`

In [ ]:
result_df = (
    min_dist_df
        .where(col("id") > disjoint_set_vertices)
        .groupBy("dist")
        .count()
)
timer(lambda: result_df.show())

## GraphLab

### Implementation

In [ ]:
def graphlab_min_dist(scope: DataFrame, values: DataFrame, steps: int) -> DataFrame:
    min_dist_df = values

    for i in range(steps):
        min_dist_nbr_df = (
            scope.alias("s")
                .join(min_dist_df.alias("r"), col("s.nbr") == col("r.id"))
                .groupBy(col("s.id"))
                .agg(F.min(F.expr("dist + weight")).alias("acc_min_dist"))
        )

        min_dist_df = (
            min_dist_nbr_df
                .join(min_dist_df, "id", "right")
                .select("id", F.least(col("dist"), col("acc_min_dist")).alias("dist"))
        )
        if i % 5 == 0:
            min_dist_df = min_dist_df.checkpoint()

    return min_dist_df

### Testing

#### Small Graph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
min_dist_df = get_min_dist_init_df(vertices_df, start=1)
min_dist_df = graphlab_min_dist(scope.inward, min_dist_df, steps=5)

In [ ]:
min_dist_df.show()

#### Large Graph

In [ ]:
disjoint_set_vertices = 500
edges_df, vertices_df = graph(disjoint_set_vertices)

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
min_dist_df = get_min_dist_init_df(vertices_df, start=1)
min_dist_df = timer(lambda: graphlab_min_dist(scope.inward, min_dist_df, steps=12))

All vertices from the same disjoint set as the vertex `1` should have a distance less that `INF (sys.maxsize)`

In [ ]:
df = min_dist_df.where(col("id") < disjoint_set_vertices).where(col("dist") < sys.maxsize)
timer(lambda: df.count())

All vertices from the other disjoint sets comparing to the vertex `1` should have the minimum distance exactly `INF (sys.maxsize)`

In [ ]:
df = min_dist_df.where(col("id") > disjoint_set_vertices).groupby("dist").count()
timer(lambda: df.show())

## PowerGraph

### Implementation

In [ ]:
def powergraph_min_dist(scope: DataFrame, values: DataFrame, steps: int):
    result_df = values

    for i in range(steps):
        gather_df = (
            scope.where("id != nbr").alias("s")
                .join(result_df.alias("r"), col("r.id") == col("s.nbr"))
                .select(col("s.id"), F.expr("r.dist + s.weight").alias("dist"))
        )
        sum_df = (
            gather_df
                .groupBy("id")
                .agg(F.min("dist").alias("acc_dist"))
        )
        apply_df = (
            sum_df
                .join(result_df, "id", "right")
                .select(col("id"), F.least("acc_dist", "dist").alias("dist"))
        )
        scatter_df = apply_df
        result_df = scatter_df
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
pg_min_dist_result_df = get_min_dist_init_df(vertices_df, start=1)
pg_min_dist_result_df = powergraph_min_dist(scope.inward, pg_min_dist_result_df, steps=5)

In [ ]:
pg_min_dist_result_df.show()

#### Large Graph

In [ ]:
disjoint_set_vertices = 500
edges_df, vertices_df = graph(disjoint_set_vertices)

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
pg_min_dist_result_df = get_min_dist_init_df(vertices_df, start=1)
pg_min_dist_result_df = timer(lambda: powergraph_min_dist(scope.inward, pg_min_dist_result_df, steps=12))

All vertices from the same disjoint set as the vertex `1` should have a distance less that `INF (sys.maxsize)`

In [ ]:
df = pg_min_dist_result_df.where(col("id") < disjoint_set_vertices).where(col("dist") < sys.maxsize)
timer(lambda: df.count())

All vertices from the other disjoint sets comparing to the vertex `1` should have the minimum distance exactly `INF (sys.maxsize)`

In [ ]:
df = pg_min_dist_result_df.where(col("id") > disjoint_set_vertices).groupby("dist").count()
timer(lambda: df.show())

# Топологическая сортировка - Topological Sort

## Pregel

### Подготовка данных

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

### Реализация

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))

for _ in range(5):
    sorted_df = (
        edges_df.where(col("src") != col("dst"))
            .join(sorted_df, col("src") == col("id"))
            .select(col("dst").alias("id"), col("ord").alias("message"))
            .groupby(col("id")).agg(F.max("message").alias("message"))
            .join(sorted_df, "id", "right")
            .select("id", F.greatest(F.expr("message + 1"), "ord").alias("ord"))
    )

sorted_df.orderBy("ord").show()

## GraphLab

### Подготовка данных

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

### Реализация I

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))
inward = scope.inward

for _ in range(5):
    sorted_nbr_df = (
        inward.where("src != dst").alias("s")
            .join(sorted_df.alias("r"), col("s.nbr") == col("r.id"))
            .select(col("s.id"), F.col("r.ord"))
            .groupBy(col("s.id"))
            .agg(F.max("ord").alias("acc_ord"))
    )
    
    sorted_df = (
        sorted_nbr_df.alias("a")
            .join(sorted_df.alias("r"), "id", "right")
            .select("id", F.greatest(col("r.ord"), F.expr("acc_ord + 1")).alias("ord"))
    )

sorted_df.orderBy("ord").show()

### Реализация II

In [ ]:
inward = scope.inward
outward = scope.outward

sorted_df = vertices_df.withColumn("ord", F.rand())

for _ in range(5):
    inward_nbr_df = (
        inward.alias("s")
            .join(sorted_df.alias("r"), col("s.nbr") == col("r.id"))
            .select(col("s.id"), F.expr("ord").alias("ord"), col("r.id"))
            .groupBy(col("s.id"))
            .agg(F.max("ord").alias("in_acc_ord"))
    )
    outward_nbr_df = (
        outward.alias("s")
            .join(sorted_df.alias("r"), col("s.nbr") == col("r.id"))
            .select(col("s.id"), F.expr("ord").alias("ord"), col("r.id"))
            .groupBy(col("s.id"))
            .agg(F.min("ord").alias("out_acc_ord"))
    )
    sorted_df = (
        inward_nbr_df.join(outward_nbr_df, "id", "full")
            .join(sorted_df, "id", "right")
            .select("id", F.coalesce("in_acc_ord", F.lit(0)).alias("max_in"), F.coalesce("out_acc_ord", F.lit(1)).alias("min_out"), "ord")
            .select("id", F.abs(F.expr("min_out - max_in")).alias("diff"), "ord")
            .select("id", F.expr("diff / 2").alias("ord"))
    )

sorted_df.orderBy("ord").show()

## PowerGraph

In [ ]:
edges_df, vertices_df = small_graph()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
scope = Scope(edges_df)

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))

inward = scope.inward
for _ in range(5):
    gather_df = (
        inward.where("src != dst").alias("s")
            .join(
                sorted_df.alias("r"),
                col("r.id") == col("s.nbr"))
            .select(col("s.id"), col("r.ord"))
    )
    sum_df = gather_df.groupBy("id").agg(F.max("ord").alias("acc_ord"))
    apply_df = (
        sum_df
            .join(sorted_df, "id", "right")
            .select(col("id"), F.greatest(F.expr("acc_ord + 1"), "ord").alias("ord"))
    )
    scatter_df = apply_df
    sorted_df = scatter_df

sorted_df.orderBy("ord").show()

# Общий алгоритм

## Pregel

In [ ]:
def pregel_superstep(edges: DataFrame, values: DataFrame, message: Column, combiner: Callable[[Column], Column], computer: Column, **columns: Column) -> DataFrame:
    message_box_df = (
        edges.where(col("src") != col("dst"))
            .join(values, col("src") == col("id"))
            .select(col("dst").alias("id"), message.alias("message"))
    )
    accumulator_df = (
        message_box_df
            .groupby(col("id"))
            .agg(combiner(col("message")).alias("message"))
    )
    return (
        accumulator_df
            .join(values, "id", "right")
            .select("id", computer.alias("value"), *columns.values())
    )

In [ ]:
edges_df, vertices_df = small_graph()

### Связные компоненты - Connected Components

In [ ]:
cc_df = vertices_df.withColumn("value", col("id"))

for _ in range(5):
    cc_df = pregel_superstep(
        edges=edges_df,
        values=cc_df,
        message=col("value"),
        combiner=F.min,
        computer=F.least(col("value"), col("message"))
    )

cc_df.show()

### Кратчайщее расстояние - Single Point Shortest Path

In [ ]:
pregel_min_dist_df = get_min_dist_init_df(start=1)

for _ in range(5):
    pregel_min_dist_df = pregel_superstep(
        edges=edges_df,
        values=pregel_min_dist_df,
        message=F.expr("value + weight"),
        combiner=F.min,
        computer=F.least("message", "value")
    )

pregel_min_dist_df.show()

### Топологическая сортировка - Topological Sort

In [ ]:
pregel_sort_df = vertices_df.withColumn("value", F.lit(1))

for _ in range(5):
    pregel_sort_df = pregel_superstep(
        edges=edges_df,
        values=pregel_sort_df,
        message=col("value"),
        combiner=F.max,
        computer=F.greatest(F.expr("message + 1"), "value")
    )

pregel_sort_df.orderBy("value").show()

### PageRank

In [ ]:
in_deg = edges_df.groupBy("dst").agg(F.count(F.lit(1)).alias("in_deg")).withColumnRenamed("dst", "id")
out_deg = edges_df.groupBy("src").agg(F.count(F.lit(1)).alias("out_deg")).withColumnRenamed("src", "id")
deg_vert_df = vertices_df.join(in_deg, "id", "left").join(out_deg, "id", "left").select("id", F.coalesce("in_deg", F.lit(1)).alias("in_deg"), F.coalesce("out_deg", F.lit(1)).alias("out_deg"))

In [ ]:
page_rank_result_df = deg_vert_df.withColumn("value", F.lit(1))
page_rank_result_df.show()

In [ ]:
for i in range(10):
    page_rank_result_df = pregel_superstep(
        edges=edges_df,
        values=page_rank_result_df,
        message=col("value"),
        combiner=F.sum,
        computer=F.expr("(0.15 + 0.85 * nvl(message, value)) / out_deg"),
        out_deg=col("out_deg")
    )
    if i % 5 == 0:
        page_rank_result_df = page_rank_result_df.checkpoint()

page_rank_result_df.show()

## GraphLab

In [ ]:
def graphlab_step(scope: DataFrame, values: DataFrame, nbr_combiner: Column, result: Column, **columns: Column):
    values_nbr_df = (
        scope.where("id != nbr").alias("s")
            .join(values.alias("r"), col("s.nbr") == col("r.id"))
            .groupBy(col("s.id"))
            .agg(nbr_combiner)
    )
    values_df = (
        values_nbr_df
            .join(values, "id", "right")
            .select("id", result, *columns.values())
    )
    return values_df

In [ ]:
edges_df, vertices_df = small_graph()
nbrs = Scope(edges_df)

### Connected Components

In [ ]:
cc_df = vertices_df.withColumn("cc", col("id"))

for _ in range(5):
    cc_df = graphlab_step(
        scope=nbrs(Scope.Direction.INOUT),
        values=cc_df,
        nbr_combiner=F.min("cc").alias("acc_cc"),
        result=F.least(col("cc"), col("acc_cc")).alias("cc")
    )

cc_df.show()

### Single Source Shortest Path

In [ ]:
min_dist_df = get_min_dist_init_df(start=1)

for _ in range(5):
    min_dist_df = graphlab_step(
        scope=nbrs(Scope.Direction.IN),
        values=min_dist_df,
        nbr_combiner=F.min(F.expr("value + weight")).alias("nbr_min_dist"),
        result=F.least(col("value"), col("nbr_min_dist")).alias("value")
    )

min_dist_df.show()

### Топологическая сортировка - Topological Sort

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))
for _ in range(5):
    sorted_df = graphlab_step(
        scope=nbrs(Scope.Direction.IN),
        values=sorted_df,
        nbr_combiner=F.max("ord").alias("acc_ord"),
        result=F.greatest(col("ord"), F.expr("acc_ord + 1")).alias("ord")
    )

sorted_df.orderBy("ord").show()

### Page Rank

In [ ]:
in_deg = edges_df.groupBy("dst").agg(F.count(F.lit(1)).alias("in_deg")).withColumnRenamed("dst", "id")
out_deg = edges_df.groupBy("src").agg(F.count(F.lit(1)).alias("out_deg")).withColumnRenamed("src", "id")
deg_vert_df = vertices_df.join(in_deg, "id", "left").join(out_deg, "id", "left").select("id", F.expr("nvl(in_deg, 1)").alias("in_deg"), F.expr("nvl(out_deg, 1)").alias("out_deg"))

In [ ]:
page_rank_result_df = deg_vert_df.withColumn("value", F.lit(1))

for i in range(10):
    page_rank_result_df = graphlab_step(
        scope=nbrs(Scope.Direction.IN),
        values=page_rank_result_df,
        nbr_combiner=F.sum("value").alias("nbr_value"),
        result=F.expr("(0.15 + 0.85 * nvl(nbr_value, value)) / out_deg").alias("value"),
        out_deg=col("out_deg")
    )    
    if i % 5 == 0:
        page_rank_result_df = page_rank_result_df.checkpoint()

page_rank_result_df.show()

## PowerGraph

In [ ]:
def powergraph_superstep(scope: DataFrame, values: DataFrame, gather: Column, combiner: Column, apply: Column, **columns: Column):
    gather_df = (
        scope.where("id != nbr").alias("s")
            .join(
                values.alias("r"),
                col("r.id") == col("s.nbr"))
            .select(col("s.id"), gather)
    )
    sum_df = gather_df.groupBy("id").agg(combiner)
    apply_df = (
        sum_df
            .join(values, "id", "right")
            .select(col("id"), apply, *columns.values())
    )
    scatter_df = apply_df
    return scatter_df

In [ ]:
edges_df, vertices_df = small_graph()
nbrs = Scope(edges_df)

### Connected Components

In [ ]:
cc_df = vertices_df.withColumn("cc", col("id"))

for _ in range(5):
    cc_df = powergraph_superstep(
        scope=nbrs(Scope.Direction.INOUT),
        values=cc_df,
        gather=col("cc"),
        combiner=F.min("cc").alias("acc_cc"),
        apply=F.least("cc", "acc_cc").alias("cc")
    )

cc_df.show()

### Single Source Shortest Path

In [ ]:
min_dist_df = get_min_dist_init_df(start=1)

for _ in range(5):
    min_dist_df = powergraph_superstep(
        scope=nbrs(Scope.Direction.IN),
        values=min_dist_df,
        gather=F.expr("value + weight").alias("dist"),
        combiner=F.min("dist").alias("acc_dist"),
        apply=F.least("acc_dist", "value").alias("value")
    )

min_dist_df.show()

### Topological Sort

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))

for _ in range(5):
    sorted_df = powergraph_superstep(
        scope=nbrs(Scope.Direction.IN),
        values=sorted_df,
        gather=col("ord"),
        combiner=F.max("ord").alias("acc_ord"),
        apply=F.greatest(F.expr("acc_ord + 1"), "ord").alias("ord")
    )

sorted_df.show()

### Page Rank

In [ ]:
in_deg = edges_df.groupBy("dst").agg(F.count(F.lit(1)).alias("in_deg")).withColumnRenamed("dst", "id")
out_deg = edges_df.groupBy("src").agg(F.count(F.lit(1)).alias("out_deg")).withColumnRenamed("src", "id")
deg_vert_df = vertices_df.join(in_deg, "id", "left").join(out_deg, "id", "left").select("id", F.expr("nvl(in_deg, 1)").alias("in_deg"), F.expr("nvl(out_deg, 1)").alias("out_deg"))

In [ ]:
page_rank_result_df = deg_vert_df.withColumn("value", F.lit(1))

for i in range(10):
    page_rank_result_df = powergraph_superstep(
        scope=nbrs(Scope.Direction.IN),
        values=page_rank_result_df,
        gather=col("value"),
        combiner=F.sum("value").alias("in_value"),
        apply=F.expr("(0.15 + 0.85 * nvl(in_value, value)) / out_deg").alias("value"),
        out_deg=col("out_deg")
    )    
    if i % 5 == 0:
        page_rank_result_df = page_rank_result_df.checkpoint()

page_rank_result_df.show()